# 1 - Insert into DB

This notebook takes CSV files and inserts them into the DB. This step is the first of the pipeline to take GPS deer tracking data and create a prediction of deer locations.

In [1]:
import pandas as pd
import geopandas as gpd
import psycopg2 as pg
import psycopg2.extras as extras
import numpy as np

## Config
The "gps_file" is the csv that contains the deer GPS data while the "individual file" contains information on the individual animals. The HOST and PORT information must match those of the DB. 

NOTE: Don't use "localhost" or "127.0.0.1" in a docker environment... you can never exactly tell what those are when you're working with containers...

In [2]:
gps_file = './data/gps/AWP_ALL_DEER_GPS_Cleaned.csv'
individual_file = './data/gps/Individuals.csv'

DO_INSERTS = True

# HOST = '10.0.0.200'
HOST = '10.152.56.176'
HOST = '10.0.0.91'
PORT = '5555'

In [3]:
individual_dtypes={'Collar':'str', 
                 'Collar ID':'str', 
                 'Deer ID':'str',
                 'Sex':'str',
                 'Age at deployment ':'float',
                 'Park':'str',
                 'Notes':'str'}

indi_df = pd.read_csv(individual_file, dtype = individual_dtypes)
# There is one column where Deer ID is blank, a collar without a deer is not valuable so all instances of it should be removed.
# indi_df['Deer ID'].isna()
indi_df = indi_df.dropna(subset=['Deer ID'])

## Insert High Res Pop into DB

In [4]:
# https://www.geeksforgeeks.org/how-to-insert-a-pandas-dataframe-to-an-existing-postgresql-table/
def execute_values(conn, df, table): 
  
    tuples = [tuple(x) for x in df.to_numpy()] 
  
    cols = ','.join(list(df.columns)) 
    # SQL query to execute 
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols) 
    cursor = conn.cursor() 
    try: 
        extras.execute_values(cursor, query, tuples) 
        conn.commit() 
    except (Exception, pg.DatabaseError) as error: 
        print("Error: %s" % error) 
        conn.rollback() 
        cursor.close() 
        return 1
    print("the dataframe is inserted") 
    cursor.close() 

Insert individuals dataframe into the Postgres DB
```SQL
CREATE TABLE IF NOT EXISTS deer.individuals
(
    deer_id	integer primary key,
    collar text,
    collar_ID  text,
    sex	 text,
    age_at_deployment int,	
    park_name  text,
    notes  text    
);
```

In [5]:
indi_column_rename = {'Collar':'collar', 
                 'Collar ID':'collar_ID', 
                 'Deer ID':'deer_id',
                 'Sex':'sex',
                 'Age at deployment ':'age_at_deployment',
                 'Park':'park_name',
                 'Notes':'notes'}

indi_df = indi_df.rename(columns=indi_column_rename) 

if DO_INSERTS:
    print('Doing Inserts')
    conn = pg.connect(database="deer", user='gmu', password='super_secret_password', host=HOST, port=PORT) 
    execute_values(conn, indi_df, 'deer.individuals') 

Doing Inserts
the dataframe is inserted


GPS table definition
```SQL
CREATE TABLE IF NOT EXISTS deer.gps
(
    event_id integer,
    "timestamp" timestamp with time zone NOT NULL,
    geom geometry(Point,4326),
    deer_id integer,
    tag_id text COLLATE pg_catalog."default",
    external_temp numeric,
    gps_dop numeric,
    height_above_ellipsoid numeric,
    CONSTRAINT gps_deer_id_fkey FOREIGN KEY (deer_id)
        REFERENCES deer.individuals (deer_id) MATCH SIMPLE
        ON UPDATE NO ACTION
        ON DELETE NO ACTION
)
```

In [6]:
gps_df = pd.read_csv(gps_file)
gps_df['datetime'] = pd.to_datetime(gps_df['Date'] + ' ' + gps_df['GMT Time Stamp'],format='%d/%m/%Y %H:%M:%S')
gps_gdf = gpd.GeoDataFrame(gps_df, 
                       geometry=gpd.points_from_xy(gps_df['location-long'], gps_df['location-lat']), 
                       crs="EPSG:4326")
gps_column_rename = {'event_id':'event_id', 
                 'datetime':'timestamp', 
                 'geometry':'geom',
                 'individual-local-identifier':'deer_id',
                 'tag-local-identifier':'tag_id',
                 'external-temperature':'external_temp',
                 'gps:dop':'gps_dop',
                 'height-above-ellipsoid': "height_above_ellipsoid" }
gps_gdf = gps_gdf[gps_column_rename.keys()]
gps_gdf = gps_gdf.rename(columns=gps_column_rename) 
gps_gdf = gps_gdf.set_geometry("geom")
 

/tmp/ipykernel_63001/1302088199.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  gps_df = pd.read_csv(gps_file)


In [7]:
from sqlalchemy import create_engine
# https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.to_postgis.html
if DO_INSERTS:
    # conn = pg.connect(database="deer", user='gmu', password='super_secret_password', host=HOST, port=PORT) 
    engine = create_engine(f"postgresql://gmu:super_secret_password@{HOST}:{PORT}/deer")  
    gps_gdf.to_postgis("gps", engine, schema = "deer" ,if_exists='append')  


In [8]:
gps_gdf

,event_id,timestamp,geom,deer_id,tag_id,external_temp,gps_dop,height_above_ellipsoid
0,21268352997,2017-02-01 00:00:10,POINT (-76.92629 39.02482),31,39925,12,3.2,13.20
1,21268352998,2017-02-01 01:01:00,POINT (-76.92636 39.02517),31,39925,11,4.2,-5.30
2,21268352999,2017-02-01 02:00:54,POINT (-76.92649 39.02505),31,39925,10,2.0,18.29
3,21268353000,2017-02-01 03:00:18,POINT (-76.92629 39.02495),31,39925,8,5.2,-16.65
4,21268353001,2017-02-01 04:01:00,POINT (-76.92661 39.02499),31,39925,6,3.2,11.45
...,...,...,...,...,...,...,...,...
296347,14091138461,2019-10-02 01:00:16,POINT (-76.75696 39.21183),50,42997,28,3.8,58.55
296348,14091138462,2019-10-02 02:00:29,POINT (-76.75807 39.21173),50,42997,27,2.2,62.74
296349,14091138463,2019-10-02 03:00:14,POINT (-76.75758 39.21361),50,42997,26,2.4,70.26
296350,14091138464,2019-10-02 04:00:16,POINT (-76.75803 39.21404),50,42997,26,3.8,118.93
